##(a)Creating If then clauses using the Diabetes Data set and emplyoing pruning to reduce the number of if then clauses

In [ ]:
from sklearn.datasets import fetch_openml as f
from sklearn.model_selection import train_test_split as s
from sklearn.tree import DecisionTreeClassifier as d, export_text as e
from sklearn.metrics import accuracy_score as a
p=f(name='diabetes',version=1)
X,y=p.data,p.target
X_train,X_test,y_train,y_test=s(X,y,test_size=0.2,random_state=42)
c=d()
c.fit(X_train,y_train)
print("If-Then Clauses:")
print(e(c,feature_names=p.feature_names))
o=c.tree_.node_count
print("\nOriginal number of if-then clauses:",o)
y_pred=c.predict(X_test)
accuracy=a(y_test,y_pred)
print("Accuracy without minimization:",accuracy)
p_c=d(max_depth=3)
p_c.fit(X_train,y_train)
p_n=p_c.tree_.node_count
print("\nPruned number of if-then clauses:",p_n)
y_pred_p=p_c.predict(X_test)
a_p=a(y_test,y_pred_p)
print("Accuracy after pruning:",a_p)
from sklearn.neural_network import MLPClassifier as m
mlp=m(hidden_layer_sizes=(100,),max_iter=1000,random_state=42)
mlp.fit(X_train,y_train)
y_pred_m=mlp.predict(X_test)
a_m=a(y_test,y_pred_m)
print("Accuracy with MLP:",a_m)


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


If-Then Clauses:
|--- plas <= 127.50
|   |--- age <= 28.50
|   |   |--- mass <= 30.95
|   |   |   |--- pedi <= 0.67
|   |   |   |   |--- class: tested_negative
|   |   |   |--- pedi >  0.67
|   |   |   |   |--- pedi <= 0.69
|   |   |   |   |   |--- class: tested_positive
|   |   |   |   |--- pedi >  0.69
|   |   |   |   |   |--- class: tested_negative
|   |   |--- mass >  30.95
|   |   |   |--- pres <= 37.00
|   |   |   |   |--- class: tested_positive
|   |   |   |--- pres >  37.00
|   |   |   |   |--- pedi <= 0.50
|   |   |   |   |   |--- skin <= 8.50
|   |   |   |   |   |   |--- plas <= 109.50
|   |   |   |   |   |   |   |--- class: tested_negative
|   |   |   |   |   |   |--- plas >  109.50
|   |   |   |   |   |   |   |--- mass <= 32.45
|   |   |   |   |   |   |   |   |--- class: tested_negative
|   |   |   |   |   |   |   |--- mass >  32.45
|   |   |   |   |   |   |   |   |--- class: tested_positive
|   |   |   |   |   |--- skin >  8.50
|   |   |   |   |   |   |--- mass <= 31.10
| 

Employing grid sreach. By using grid search, we systematically explore different values of max_depth to find the one that optimizes model performance. This process allows us to find the optimal balance between model complexity and generalization.

In [ ]:
from sklearn.datasets import fetch_openml as f
from sklearn.model_selection import train_test_split as s, GridSearchCV as g
from sklearn.tree import DecisionTreeClassifier as d, export_text as e
from sklearn.metrics import accuracy_score as a

p = f(name='diabetes', version=1)
X, y = p.data, p.target
X_tr, X_te, y_tr, y_te = s(X, y, test_size=0.2, random_state=42)

param_grid = {'max_depth': range(1, 11)}

grid_s = g(d(), param_grid, cv=5)
grid_s.fit(X_tr, y_tr)

best_c = grid_s.best_estimator_

print("If-Then Clauses:")
print(e(best_c, feature_names=p.feature_names))

y_pred_b = best_c.predict(X_te)
accuracy_b = a(y_te, y_pred_b)
print("Accuracy:", accuracy_b)


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


If-Then Clauses:
|--- plas <= 127.50
|   |--- age <= 28.50
|   |   |--- mass <= 30.95
|   |   |   |--- class: tested_negative
|   |   |--- mass >  30.95
|   |   |   |--- class: tested_negative
|   |--- age >  28.50
|   |   |--- mass <= 26.35
|   |   |   |--- class: tested_negative
|   |   |--- mass >  26.35
|   |   |   |--- class: tested_negative
|--- plas >  127.50
|   |--- mass <= 29.95
|   |   |--- plas <= 146.50
|   |   |   |--- class: tested_negative
|   |   |--- plas >  146.50
|   |   |   |--- class: tested_positive
|   |--- mass >  29.95
|   |   |--- plas <= 165.50
|   |   |   |--- class: tested_positive
|   |   |--- plas >  165.50
|   |   |   |--- class: tested_positive

Accuracy: 0.7597402597402597


Method 3 : Using feature selection

In [ ]:
from sklearn.datasets import fetch_openml as f
from sklearn.model_selection import train_test_split as s
from sklearn.tree import DecisionTreeClassifier as d, export_text as e
from sklearn.feature_selection import SelectFromModel as m
from sklearn.metrics import accuracy_score as a

p = f(name='diabetes', version=1)
X, y = p.data, p.target
X_tr, X_te, y_tr, y_te = s(X, y, test_size=0.2, random_state=42)

clf = d()
clf.fit(X_tr, y_tr)

print("Number of If-Then Clauses before feature selection:", clf.tree_.node_count)

feature_selector = m(clf, prefit=True)
X_tr_sel = feature_selector.transform(X_tr)
X_te_sel = feature_selector.transform(X_te)

sel_clf = d()
sel_clf.fit(X_tr_sel, y_tr)

print("Number of If-Then Clauses after feature selection:", sel_clf.tree_.node_count)

y_pred_sel = sel_clf.predict(X_te_sel)
accuracy_sel = a(y_te, y_pred_sel)
print("Accuracy after feature selection:", accuracy_sel)


Number of If-Then Clauses before feature selection: 205
Number of If-Then Clauses after feature selection: 323
Accuracy after feature selection: 0.6623376623376623


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


We can observe that the pruning technique proved to be the best for the Diabetes data set


##(b) Emplyoing same methods on different data sets, starting with the heart data set

In [ ]:
from sklearn.datasets import fetch_openml as f
from sklearn.model_selection import train_test_split as s
from sklearn.tree import DecisionTreeClassifier as d, export_text as e
from sklearn.metrics import accuracy_score as a

heart = f(name='heart', version=1)
X, y = heart.data, heart.target
X_tr, X_te, y_tr, y_te = s(X, y, test_size=0.2, random_state=42)

clf = d()
clf.fit(X_tr, y_tr)

print("If-Then Clauses:")
print(e(clf, feature_names=heart.feature_names))

original_clauses = clf.tree_.node_count
print("\nOriginal number of if-then clauses:", original_clauses)

y_pred = clf.predict(X_te)
accuracy = a(y_te, y_pred)
print("Accuracy without minimization:", accuracy)

pruned_clf = d(max_depth=3)
pruned_clf.fit(X_tr, y_tr)

pruned_clauses = pruned_clf.tree_.node_count
print("\nPruned number of if-then clauses:", pruned_clauses)

y_pred_pruned = pruned_clf.predict(X_te)
accuracy_pruned = a(y_te, y_pred_pruned)
print("Accuracy after pruning:", accuracy_pruned)


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


If-Then Clauses:
|--- att_3 <= 0.67
|   |--- att_10 <= -0.21
|   |   |--- att_13 <= -0.25
|   |   |   |--- att_5 <= -0.33
|   |   |   |   |--- att_4 <= -0.72
|   |   |   |   |   |--- att_5 <= -0.51
|   |   |   |   |   |   |--- class: -1.0
|   |   |   |   |   |--- att_5 >  -0.51
|   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |--- att_4 >  -0.72
|   |   |   |   |   |--- class: -1.0
|   |   |   |--- att_5 >  -0.33
|   |   |   |   |--- att_10 <= -0.56
|   |   |   |   |   |--- att_8 <= 0.02
|   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |   |--- att_8 >  0.02
|   |   |   |   |   |   |--- att_5 <= -0.07
|   |   |   |   |   |   |   |--- class: -1.0
|   |   |   |   |   |   |--- att_5 >  -0.07
|   |   |   |   |   |   |   |--- att_5 <= -0.03
|   |   |   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |   |   |   |--- att_5 >  -0.03
|   |   |   |   |   |   |   |   |--- class: -1.0
|   |   |   |   |--- att_10 >  -0.56
|   |   |   |   |   |--- class: 1.0
|   |   |--- att_13 > 

Trying grid search.Defining a parameter grid param_grid containing different values for max_depth, min_samples_split, and min_samples_leaf.
Use GridSearchCV to perform grid search with 5-fold cross-validation.
We fit the grid search object on the training data.
Finally, we evaluate the accuracy on the test set using the best estimator.
This approach allows us to tune hyperparameters using grid search

In [ ]:
from sklearn.datasets import fetch_openml as f
from sklearn.model_selection import train_test_split as s, GridSearchCV as g
from sklearn.tree import DecisionTreeClassifier as d, export_text as e
from sklearn.metrics import accuracy_score as a

heart = f(name='heart', version=1)
X, y = heart.data, heart.target
X_tr, X_te, y_tr, y_te = s(X, y, test_size=0.2, random_state=42)

param_grid = {
    'max_depth': [3, 5, 7, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

clf = d()

grid_search = g(clf, param_grid, cv=5)

grid_search.fit(X_tr, y_tr)

best_clf = grid_search.best_estimator_

print("Best parameters found by grid search:", grid_search.best_params_)

print("\nIf-Then Clauses of the Best Estimator:")
print(e(best_clf, feature_names=heart.feature_names))

y_pred_best = best_clf.predict(X_te)
accuracy_best = a(y_te, y_pred_best)
print("\nAccuracy using the best estimator:", accuracy_best)


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Best parameters found by grid search: {'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 10}

If-Then Clauses of the Best Estimator:
|--- att_3 <= 0.67
|   |--- att_10 <= -0.21
|   |   |--- att_13 <= -0.25
|   |   |   |--- class: -1.0
|   |   |--- att_13 >  -0.25
|   |   |   |--- class: -1.0
|   |--- att_10 >  -0.21
|   |   |--- class: 1.0
|--- att_3 >  0.67
|   |--- att_10 <= -0.82
|   |   |--- att_12 <= -0.67
|   |   |   |--- class: -1.0
|   |   |--- att_12 >  -0.67
|   |   |   |--- class: 1.0
|   |--- att_10 >  -0.82
|   |   |--- att_13 <= 0.75
|   |   |   |--- class: 1.0
|   |   |--- att_13 >  0.75
|   |   |   |--- class: 1.0


Accuracy using the best estimator: 0.7222222222222222


Trying to reduce the number of if then clauses using feature selection ( reducing nuber of dimensions )

In [ ]:
from sklearn.datasets import fetch_openml as f
from sklearn.model_selection import train_test_split as s
from sklearn.tree import DecisionTreeClassifier as d, export_text as e
from sklearn.metrics import accuracy_score as a
from sklearn.feature_selection import SelectKBest as k, f_classif as fc

heart = f(name='heart', version=1)
X, y = heart.data, heart.target

orig_nf = X.shape[1]

num_feat = 5
selector = k(score_func=fc, k=num_feat)
X_sel = selector.fit_transform(X, y)

sel_feat_ind = selector.get_support(indices=True)

X_tr, X_te, y_tr, y_te = s(X_sel, y, test_size=0.2, random_state=42)

clf_b = d()
clf_b.fit(X_tr, y_tr)

orig_clauses = clf_b.tree_.node_count

print("Original number of if-then clauses before feature selection:", orig_clauses)

clf_a = d()
clf_a.fit(X_sel, y)

print("If-Then Clauses after feature selection:")
sel_feat_names = [heart.feature_names[i] for i in sel_feat_ind]
if_then_text = e(clf_a, feature_names=sel_feat_names)
print(if_then_text)

clauses_aft = clf_a.tree_.node_count
print("\nNumber of if-then clauses after feature selection:", clauses_aft)

y_pred_aft = clf_a.predict(X_te)
accuracy_aft = a(y_te, y_pred_aft)
print("Accuracy after feature selection:", accuracy_aft)


Original number of if-then clauses before feature selection: 113
If-Then Clauses after feature selection:
|--- att_13 <= -0.25
|   |--- att_12 <= -0.67
|   |   |--- att_10 <= -0.10
|   |   |   |--- att_8 <= -0.81
|   |   |   |   |--- class: 1.0
|   |   |   |--- att_8 >  -0.81
|   |   |   |   |--- att_8 <= 0.38
|   |   |   |   |   |--- att_10 <= -0.98
|   |   |   |   |   |   |--- att_8 <= 0.37
|   |   |   |   |   |   |   |--- att_8 <= 0.24
|   |   |   |   |   |   |   |   |--- att_8 <= -0.40
|   |   |   |   |   |   |   |   |   |--- class: -1.0
|   |   |   |   |   |   |   |   |--- att_8 >  -0.40
|   |   |   |   |   |   |   |   |   |--- att_8 <= -0.08
|   |   |   |   |   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |   |   |   |   |   |--- att_8 >  -0.08
|   |   |   |   |   |   |   |   |   |   |--- att_8 <= 0.08
|   |   |   |   |   |   |   |   |   |   |   |--- class: -1.0
|   |   |   |   |   |   |   |   |   |   |--- att_8 >  0.08
|   |   |   |   |   |   |   |   |   |   |   |--- trun

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Next , we employ the same methods on the Breast cancer dataset

In [ ]:
from sklearn.datasets import load_breast_cancer as lbc
from sklearn.model_selection import train_test_split as s
from sklearn.tree import DecisionTreeClassifier as d, export_text as e
from sklearn.metrics import accuracy_score as a

bc = lbc()

X_tr, X_te, y_tr, y_te = s(bc.data, bc.target, test_size=0.2, random_state=42)

clf = d()
clf.fit(X_tr, y_tr)

print("If-Then Clauses:")
print(e(clf, feature_names=bc.feature_names.tolist()))

orig_clauses = clf.tree_.node_count
print("\nOriginal number of if-then clauses:", orig_clauses)

y_pred = clf.predict(X_te)
accuracy = a(y_te, y_pred)
print("Accuracy without minimization:", accuracy)

pruned_clf = d(max_depth=3)
pruned_clf.fit(X_tr, y_tr)

pruned_clauses = pruned_clf.tree_.node_count
print("\nPruned number of if-then clauses:", pruned_clauses)

y_pred_pruned = pruned_clf.predict(X_te)
accuracy_pruned = a(y_te, y_pred_pruned)
print("Accuracy after pruning:", accuracy_pruned)


If-Then Clauses:
|--- mean concave points <= 0.05
|   |--- worst radius <= 16.83
|   |   |--- area error <= 48.70
|   |   |   |--- worst smoothness <= 0.18
|   |   |   |   |--- smoothness error <= 0.00
|   |   |   |   |   |--- mean texture <= 19.90
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- mean texture >  19.90
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |--- smoothness error >  0.00
|   |   |   |   |   |--- worst texture <= 33.35
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- worst texture >  33.35
|   |   |   |   |   |   |--- worst texture <= 33.56
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |--- worst texture >  33.56
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |--- worst smoothness >  0.18
|   |   |   |   |--- class: 0
|   |   |--- area error >  48.70
|   |   |   |--- mean compactness <= 0.06
|   |   |   |   |--- class: 0
|   |   |   |--- mean compactness >  0.06
|   |   |   |   |--- class: 1
|   |--- wo

In [ ]:
from sklearn.datasets import load_breast_cancer as lbc
from sklearn.model_selection import train_test_split as s
from sklearn.tree import DecisionTreeClassifier as d, export_text as e
from sklearn.metrics import accuracy_score as a
from sklearn.feature_selection import SelectKBest as k, f_classif as fc

bc = lbc()

X, y = bc.data, bc.target

num_feat_select = 10
selector = k(score_func=fc, k=num_feat_select)
X_sel = selector.fit_transform(X, y)

sel_feat_indices = selector.get_support(indices=True)

X_tr, X_te, y_tr, y_te = s(X_sel, y, test_size=0.2, random_state=42)

clf = d()
clf.fit(X_tr, y_tr)

print("If-Then Clauses after feature selection:")
sel_feat_names = [bc.feature_names[i] for i in sel_feat_indices]
print(e(clf, feature_names=sel_feat_names))

clauses_aft_fs = clf.tree_.node_count
print("\nNumber of if-then clauses after feature selection:", clauses_aft_fs)

y_pred_aft_fs = clf.predict(X_te)
accuracy_aft_fs = a(y_te, y_pred_aft_fs)
print("Accuracy after feature selection:", accuracy_aft_fs)


If-Then Clauses after feature selection:
|--- mean concave points <= 0.05
|   |--- worst radius <= 16.83
|   |   |--- mean area <= 696.25
|   |   |   |--- worst concave points <= 0.14
|   |   |   |   |--- worst concave points <= 0.09
|   |   |   |   |   |--- mean concave points <= 0.03
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- mean concave points >  0.03
|   |   |   |   |   |   |--- worst concave points <= 0.08
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |--- worst concave points >  0.08
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |--- worst concave points >  0.09
|   |   |   |   |   |--- worst concave points <= 0.09
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- worst concave points >  0.09
|   |   |   |   |   |   |--- mean concavity <= 0.03
|   |   |   |   |   |   |   |--- mean radius <= 13.03
|   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |--- mean radius >  13.03
|   |   |   |   |   

## (c) Artificially Generated Dataset

Using a random dataset with

Number of Instances: 100
Columns: 5

In [ ]:
from sklearn.model_selection import train_test_split as s
from sklearn.tree import DecisionTreeClassifier as d, export_text as e
from sklearn.metrics import accuracy_score as a
from sklearn.neural_network import MLPClassifier as m
import numpy as np

np.random.seed(42)
X = np.random.rand(100, 5)
y = np.random.randint(0, 2, size=100)

X_tr, X_te, y_tr, y_te = s(X, y, test_size=0.2, random_state=42)

c = d()
c.fit(X_tr, y_tr)

print("If-Then Clauses:")
print(e(c))

oc = c.tree_.node_count
print("\nOriginal number of if-then clauses:", oc)

y_p = c.predict(X_te)
acc = a(y_te, y_p)
print("Accuracy without minimization:", acc)

pc = d(max_depth=3)
pc.fit(X_tr, y_tr)

pcn = pc.tree_.node_count
print("\nPruned number of if-then clauses:", pcn)

yp_p = pc.predict(X_te)
ap = a(y_te, yp_p)
print("Accuracy after pruning:", ap)

mc = m(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
mc.fit(X_tr, y_tr)

yp_m = mc.predict(X_te)
am = a(y_te, yp_m)
print("Accuracy with MLP:", am)


If-Then Clauses:
|--- feature_2 <= 0.03
|   |--- class: 0
|--- feature_2 >  0.03
|   |--- feature_2 <= 0.46
|   |   |--- feature_1 <= 0.64
|   |   |   |--- feature_4 <= 0.25
|   |   |   |   |--- class: 0
|   |   |   |--- feature_4 >  0.25
|   |   |   |   |--- feature_2 <= 0.33
|   |   |   |   |   |--- feature_3 <= 0.90
|   |   |   |   |   |   |--- feature_3 <= 0.36
|   |   |   |   |   |   |   |--- feature_4 <= 0.39
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |--- feature_4 >  0.39
|   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- feature_3 >  0.36
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- feature_3 >  0.90
|   |   |   |   |   |   |--- feature_3 <= 0.96
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- feature_3 >  0.96
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |--- feature_2 >  0.33
|   |   |   |   |   |--- class: 1
|   |   |--- feature_1 >  0.64
|   |   |   |--- feature

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Using a random dataset with

Number of Instances: 1000
Columns: 10

In [ ]:
from sklearn.model_selection import train_test_split as s
from sklearn.tree import DecisionTreeClassifier as d, export_text as e
from sklearn.metrics import accuracy_score as a
from sklearn.neural_network import MLPClassifier as m
import numpy as np

np.random.seed(42)
X = np.random.rand(1000, 10)
y = np.random.randint(0, 2, size=1000)

X_tr, X_te, y_tr, y_te = s(X, y, test_size=0.2, random_state=42)

c = d()
c.fit(X_tr, y_tr)

print("If-Then Clauses:")
print(e(c))

oc = c.tree_.node_count
print("\nOriginal number of if-then clauses:", oc)

y_p = c.predict(X_te)
acc = a(y_te, y_p)
print("Accuracy without minimization:", acc)

pc = d(max_depth=3)
pc.fit(X_tr, y_tr)

pcn = pc.tree_.node_count
print("\nPruned number of if-then clauses:", pcn)

yp_p = pc.predict(X_te)
ap = a(y_te, yp_p)
print("Accuracy after pruning:", ap)

mc = m(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
mc.fit(X_tr, y_tr)

yp_m = mc.predict(X_te)
am = a(y_te, yp_m)
print("Accuracy with MLP:", am)


If-Then Clauses:
|--- feature_7 <= 0.82
|   |--- feature_5 <= 0.72
|   |   |--- feature_9 <= 0.02
|   |   |   |--- class: 1
|   |   |--- feature_9 >  0.02
|   |   |   |--- feature_5 <= 0.30
|   |   |   |   |--- feature_6 <= 0.48
|   |   |   |   |   |--- feature_6 <= 0.06
|   |   |   |   |   |   |--- feature_9 <= 0.22
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- feature_9 >  0.22
|   |   |   |   |   |   |   |--- feature_7 <= 0.30
|   |   |   |   |   |   |   |   |--- feature_9 <= 0.47
|   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |   |--- feature_9 >  0.47
|   |   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |--- feature_7 >  0.30
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- feature_6 >  0.06
|   |   |   |   |   |   |--- feature_8 <= 0.20
|   |   |   |   |   |   |   |--- feature_1 <= 0.53
|   |   |   |   |   |   |   |   |--- feature_5 <= 0.01
|   |   |   |   |   |   |   |   |   |-

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Using Grid Search on
Number of Instances: 100
Columns: 5

In [ ]:
from sklearn.model_selection import train_test_split as s, GridSearchCV as g
from sklearn.tree import DecisionTreeClassifier as d, export_text as e
from sklearn.metrics import accuracy_score as a
from sklearn.neural_network import MLPClassifier as m
import numpy as np

np.random.seed(42)
X = np.random.rand(100, 5)
y = np.random.randint(0, 2, size=100)

X_tr, X_te, y_tr, y_te = s(X, y, test_size=0.2, random_state=42)

p = {'max_depth': [None, 3, 5, 10]}
c = d()
gs = g(c, p, cv=5)
gs.fit(X_tr, y_tr)

bc = gs.best_estimator_

print("Best parameters for Decision Tree Classifier:", gs.best_params_)
print("If-Then Clauses for Best Decision Tree Classifier:")
print(e(bc))

y_p = bc.predict(X_te)
acc = a(y_te, y_p)
print("Accuracy with Best Decision Tree Classifier:", acc)


Best parameters for Decision Tree Classifier: {'max_depth': 3}
If-Then Clauses for Best Decision Tree Classifier:
|--- feature_2 <= 0.03
|   |--- class: 0
|--- feature_2 >  0.03
|   |--- feature_2 <= 0.46
|   |   |--- feature_1 <= 0.64
|   |   |   |--- class: 0
|   |   |--- feature_1 >  0.64
|   |   |   |--- class: 1
|   |--- feature_2 >  0.46
|   |   |--- feature_1 <= 0.13
|   |   |   |--- class: 1
|   |   |--- feature_1 >  0.13
|   |   |   |--- class: 0

Accuracy with Best Decision Tree Classifier: 0.45


Using Grid Search on
Number of Instances: 1000
Columns: 10

In [ ]:
from sklearn.model_selection import train_test_split as s, GridSearchCV as g
from sklearn.tree import DecisionTreeClassifier as d, export_text as e
from sklearn.metrics import accuracy_score as a
from sklearn.neural_network import MLPClassifier as m
import numpy as np

np.random.seed(42)
X = np.random.rand(1000, 10)
y = np.random.randint(0, 2, size=1000)

X_tr, X_te, y_tr, y_te = s(X, y, test_size=0.2, random_state=42)

param_grid_dt = {'max_depth': [None, 3, 5, 10]}
c = d()
gs = g(c, param_grid_dt, cv=5)
gs.fit(X_tr, y_tr)

bc = gs.best_estimator_

print("Best parameters for Decision Tree Classifier:", gs.best_params_)
print("If-Then Clauses for Best Decision Tree Classifier:")
print(e(bc))

y_pred = bc.predict(X_te)
acc = a(y_te, y_pred)
print("Accuracy with Best Decision Tree Classifier:", acc)


Best parameters for Decision Tree Classifier: {'max_depth': 5}
If-Then Clauses for Best Decision Tree Classifier:
|--- feature_7 <= 0.82
|   |--- feature_5 <= 0.72
|   |   |--- feature_9 <= 0.02
|   |   |   |--- class: 1
|   |   |--- feature_9 >  0.02
|   |   |   |--- feature_5 <= 0.30
|   |   |   |   |--- feature_6 <= 0.48
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- feature_6 >  0.48
|   |   |   |   |   |--- class: 0
|   |   |   |--- feature_5 >  0.30
|   |   |   |   |--- feature_2 <= 0.02
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- feature_2 >  0.02
|   |   |   |   |   |--- class: 0
|   |--- feature_5 >  0.72
|   |   |--- feature_3 <= 0.20
|   |   |   |--- feature_4 <= 0.72
|   |   |   |   |--- feature_7 <= 0.05
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- feature_7 >  0.05
|   |   |   |   |   |--- class: 0
|   |   |   |--- feature_4 >  0.72
|   |   |   |   |--- feature_8 <= 0.50
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- feature_8 >  0.50


Using Feature Selection on
Number of Instances: 100
Columns: 5

In [ ]:
from sklearn.model_selection import train_test_split as s
from sklearn.tree import DecisionTreeClassifier as d, export_text as e
from sklearn.metrics import accuracy_score as a
from sklearn.neural_network import MLPClassifier as m
from sklearn.feature_selection import SelectFromModel as f
import numpy as np

np.random.seed(42)
X = np.random.rand(100, 5)
y = np.random.randint(0, 2, size=100)

X_tr, X_te, y_tr, y_te = s(X, y, test_size=0.2, random_state=42)

fs = f(d(max_depth=3))
X_tr_s = fs.fit_transform(X_tr, y_tr)
X_te_s = fs.transform(X_te)

clf_s = d()
clf_s.fit(X_tr_s, y_tr)

print("If-Then Clauses for Selected Features:")
print(e(clf_s))

y_p_s = clf_s.predict(X_te_s)
acc_s = a(y_te, y_p_s)
print("Accuracy after feature selection:", acc_s)


If-Then Clauses for Selected Features:
|--- feature_1 <= 0.03
|   |--- class: 0
|--- feature_1 >  0.03
|   |--- feature_1 <= 0.46
|   |   |--- feature_0 <= 0.64
|   |   |   |--- feature_1 <= 0.24
|   |   |   |   |--- feature_1 <= 0.07
|   |   |   |   |   |--- feature_0 <= 0.57
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- feature_0 >  0.57
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |--- feature_1 >  0.07
|   |   |   |   |   |--- feature_0 <= 0.26
|   |   |   |   |   |   |--- feature_0 <= 0.16
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |--- feature_0 >  0.16
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- feature_0 >  0.26
|   |   |   |   |   |   |--- class: 0
|   |   |   |--- feature_1 >  0.24
|   |   |   |   |--- feature_0 <= 0.46
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- feature_0 >  0.46
|   |   |   |   |   |--- feature_0 <= 0.57
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- feature_0

Using Feature Selection on
Number of Instances: 1000
Columns: 10

In [ ]:
from sklearn.model_selection import train_test_split as s
from sklearn.tree import DecisionTreeClassifier as d, export_text as e
from sklearn.metrics import accuracy_score as a
from sklearn.neural_network import MLPClassifier as m
from sklearn.feature_selection import SelectFromModel as f
import numpy as np

np.random.seed(42)
X = np.random.rand(1000, 10)
y = np.random.randint(0, 2, size=1000)

X_tr, X_te, y_tr, y_te = s(X, y, test_size=0.3, random_state=42)

fs = f(d(max_depth=3))
X_tr_s = fs.fit_transform(X_tr, y_tr)
X_te_s = fs.transform(X_te)

clf_s = d()
clf_s.fit(X_tr_s, y_tr)

print("If-Then Clauses for Selected Features:")
print(e(clf_s))

y_p_s = clf_s.predict(X_te_s)
acc_s = a(y_te, y_p_s)
print("Accuracy after feature selection:", acc_s)


If-Then Clauses for Selected Features:
|--- feature_2 <= 0.81
|   |--- feature_3 <= 0.04
|   |   |--- feature_2 <= 0.04
|   |   |   |--- class: 0
|   |   |--- feature_2 >  0.04
|   |   |   |--- feature_3 <= 0.01
|   |   |   |   |--- class: 0
|   |   |   |--- feature_3 >  0.01
|   |   |   |   |--- class: 1
|   |--- feature_3 >  0.04
|   |   |--- feature_2 <= 0.02
|   |   |   |--- feature_3 <= 0.55
|   |   |   |   |--- class: 1
|   |   |   |--- feature_3 >  0.55
|   |   |   |   |--- feature_1 <= 0.66
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- feature_1 >  0.66
|   |   |   |   |   |--- class: 0
|   |   |--- feature_2 >  0.02
|   |   |   |--- feature_1 <= 0.72
|   |   |   |   |--- feature_1 <= 0.24
|   |   |   |   |   |--- feature_1 <= 0.05
|   |   |   |   |   |   |--- feature_0 <= 0.60
|   |   |   |   |   |   |   |--- feature_0 <= 0.30
|   |   |   |   |   |   |   |   |--- feature_0 <= 0.05
|   |   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |   |--- f